In [4]:
import  numpy as np
import pandas as pd

print(pd.Timestamp("2023-10-15 08:30:00"))
print(pd.Period("2023-10", freq='M'))
print(pd.date_range(start="2023-01-01", periods=3, freq='D'))

2023-10-15 08:30:00
2023-10
DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03'], dtype='datetime64[ns]', freq='D')


In [8]:
dates = pd.date_range("2023-10-01", periods=3, freq='D')
temps = [22, 24, 19]
dFrame1 = pd.DataFrame({"Date": dates, "Temperature": temps})
print(dFrame1)
dFrame1["Yesterday"] = dFrame1["Temperature"].shift(1)  
print(dFrame1)


dFrame1["Date"] = pd.to_datetime(dFrame1["Date"])  
dFrame1 = dFrame1.set_index("Date")  # Now index is a DateTimeIndex!  
print(dFrame1)



        Date  Temperature
0 2023-10-01           22
1 2023-10-02           24
2 2023-10-03           19
        Date  Temperature  Yesterday
0 2023-10-01           22        NaN
1 2023-10-02           24       22.0
2 2023-10-03           19       24.0
            Temperature  Yesterday
Date                              
2023-10-01           22        NaN
2023-10-02           24       22.0
2023-10-03           19       24.0


In [11]:
oct_data = dFrame1.loc["2023-10"]  
print(oct_data)
monthly_avg = dFrame1.resample("ME").mean()
print(monthly_avg["Temperature"])
monthly_avg = dFrame1.resample("M").mean()
print(monthly_avg["Temperature"])


            Temperature  Yesterday
Date                              
2023-10-01           22        NaN
2023-10-02           24       22.0
2023-10-03           19       24.0
Date
2023-10-31    21.666667
Freq: ME, Name: Temperature, dtype: float64
Date
2023-10-31    21.666667
Freq: ME, Name: Temperature, dtype: float64


C:\Users\pavan\AppData\Local\Temp\ipykernel_25380\3969677407.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_avg = dFrame1.resample("M").mean()


In [12]:
dFrame1["Hour"] = dFrame1.index.hour      # Extract hour  
dFrame1["DayOfWeek"] = dFrame1.index.dayofweek  # Monday=0, Sunday=6 
print(dFrame1)


            Temperature  Yesterday  Hour  DayOfWeek
Date                                               
2023-10-01           22        NaN     0          6
2023-10-02           24       22.0     0          0
2023-10-03           19       24.0     0          1


In [13]:
pd_td = pd.Timedelta("2 days 3:30:15")
print(pd_td)

py_td = pd_td.to_pytimedelta()
print(py_td)
print(type(py_td))


2 days 03:30:15
2 days, 3:30:15
<class 'datetime.timedelta'>


In [14]:
timestamp = pd.Timestamp("2023-10-01")
print(timestamp + pd.Timedelta(hours=6))
print(pd.Timestamp("2023-10-03") - timestamp)


2023-10-01 06:00:00
2 days 00:00:00


In [15]:
td = pd.Timedelta("3 hours 29 min 45 sec")
print(td.round("h"))
print(td.round("15min"))


0 days 03:00:00
0 days 03:30:00


In [16]:
timestamp = pd.Timestamp("2023-10-01")
print(timestamp + pd.Timedelta(hours=6))
print(pd.Timestamp("2023-10-03") - timestamp)


2023-10-01 06:00:00
2 days 00:00:00


In [17]:

events = {
    "Get up": "07:00",
    "Breakfast": "09:00",
    "Working (1st)": "09:30",
    "Tea (1st)": "11:15",
    "Working (2nd)": "11:30",
    "Lunch": "14:00",  # 2 PM
    "Working (3rd)": "14:30",
    "Tea (2nd)": "16:30",
    "Working (4th)": "16:45",
    "Leave office": "18:30",
    "Social": "19:00",
    "Dinner": "21:30",
    "Sleep": "22:30"
}

# Convert to TimedeltaIndex (durations since midnight)
time_gaps = pd.to_timedelta([t + ":00" for t in events.values()])
df = pd.DataFrame(
    {"Activity": events.keys()},
    index=pd.TimedeltaIndex(time_gaps, name="TimeSinceMidnight")
)

print(df)


                        Activity
TimeSinceMidnight               
0 days 07:00:00           Get up
0 days 09:00:00        Breakfast
0 days 09:30:00    Working (1st)
0 days 11:15:00        Tea (1st)
0 days 11:30:00    Working (2nd)
0 days 14:00:00            Lunch
0 days 14:30:00    Working (3rd)
0 days 16:30:00        Tea (2nd)
0 days 16:45:00    Working (4th)
0 days 18:30:00     Leave office
0 days 19:00:00           Social
0 days 21:30:00           Dinner
0 days 22:30:00            Sleep


In [18]:
df["TimeSpent"] = df.index.to_series().diff().shift(-1)
print(df[["Activity", "TimeSpent"]])


                        Activity       TimeSpent
TimeSinceMidnight                               
0 days 07:00:00           Get up 0 days 02:00:00
0 days 09:00:00        Breakfast 0 days 00:30:00
0 days 09:30:00    Working (1st) 0 days 01:45:00
0 days 11:15:00        Tea (1st) 0 days 00:15:00
0 days 11:30:00    Working (2nd) 0 days 02:30:00
0 days 14:00:00            Lunch 0 days 00:30:00
0 days 14:30:00    Working (3rd) 0 days 02:00:00
0 days 16:30:00        Tea (2nd) 0 days 00:15:00
0 days 16:45:00    Working (4th) 0 days 01:45:00
0 days 18:30:00     Leave office 0 days 00:30:00
0 days 19:00:00           Social 0 days 02:30:00
0 days 21:30:00           Dinner 0 days 01:00:00
0 days 22:30:00            Sleep             NaT


In [19]:

working_time = df[df["Activity"].str.contains("Working")]["TimeSpent"].sum()
print(f"Total working time: {working_time}")


Total working time: 0 days 08:00:00


In [20]:

activity_counts = df.resample("2h").count()
print(activity_counts[["Activity"]])


                   Activity
TimeSinceMidnight          
0 days 07:00:00           1
0 days 09:00:00           2
0 days 11:00:00           2
0 days 13:00:00           2
0 days 15:00:00           2
0 days 17:00:00           1
0 days 19:00:00           1
0 days 21:00:00           2
